# Esercitazione 1

Misurazione dell’overlap lessicale tra una serie di definizioni per concetti generici/specifici e concreti/astratti. Partendo dai dati sulle definizioni (presente nella cartella "dati" su Moodle), si richiede di calcolare la similarità 2-a-2 tra le definizioni (ad es. usando la cardinalità dell'intersezione dei lemmi normalizzata sulla lunghezza minima delle definizioni), aggregando (ed effettuando la media degli score di similarità) sulle due dimensioni (concretezza / specificità). 

In [17]:
import pandas as pd
import nltk
import string
from nltk.stem import PorterStemmer
import itertools
from nltk.corpus import stopwords

corpus = pd.read_csv('definizioni.tsv', sep='\t', engine='python')
corpus.head(5)

,door,ladybug,pain,blurriness
0,"A construction used to divide two rooms, tempo...","small flying insect, typically red with black ...",A feeling of physical or mental distress,sight out of focus
1,"It's an opening, it can be opened or closed.","It is an insect, it has wings, red with black ...","It is a feeling, physical or emotional. It is ...","It is the absence of definite borders, shapele..."
2,"An object that divide two room, closing an hol...",An insect that can fly. It has red or orange c...,A felling that couscious beings can experince ...,A sensation felt when you can't see clearly th...
3,Usable for access from one area to another,Small insect with a red back,Concept that describes a suffering living being,Lack of sharpness
4,Structure that delimits an area and allows acc...,Small round flying insect,Feeling of physical discomfort,Characteristic of lack of clarity or precision


In [18]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def normalize(sentence):
    tokens = nltk.word_tokenize(sentence)
    tokens = [token for token in tokens if token not in string.punctuation] #tolgo la punteggiatura
    tokens = [token.lower() for token in tokens] # sostituisco le maiuscole con le minuscole
    tokens = [token for token in tokens if token not in stop_words] # rimuovo le stop words 
    tokens = [stemmer.stem(token) for token in tokens] # stemmizzo
    
    return tokens

In [19]:
#foreach coloumn for each row in corpus apply normalize
corpus['door'] = corpus['door'].apply(normalize)
corpus['ladybug'] = corpus['ladybug'].apply(normalize)
corpus['pain'] = corpus['pain'].apply(normalize)
corpus['blurriness'] = corpus['blurriness'].apply(normalize)

## Similarità 2-a-2

In [20]:
def compare(sentence1, sentence2):
    sentence1 = set(sentence1)
    sentence2 = set(sentence2)

    return len(sentence1.intersection(sentence2)) / min(len(sentence1), len(sentence2))

In [21]:
for coloumn in corpus:
    sum = 0
    count = 0
    for sentence1, sentence2 in itertools.combinations(corpus[coloumn], 2):
        sum += compare(sentence1, sentence2)
        count += 1

    print(coloumn, ': ', sum/count)

door :  0.2260206396413296
ladybug :  0.5842484782139964
pain :  0.2423645320197044
blurriness :  0.07986849676504847


## Aggregazione

In [22]:
new_corpus = pd.DataFrame()

new_corpus['door_ladybug'] = corpus['door'].append(corpus['ladybug'])
new_corpus['pain_blurriness'] = corpus['pain'].append(corpus['blurriness'])

new_corpus = new_corpus.reset_index(drop=True)

C:\Users\OliverioM\AppData\Local\Temp\ipykernel_11140\2196199081.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_corpus['door_ladybug'] = corpus['door'].append(corpus['ladybug'])
C:\Users\OliverioM\AppData\Local\Temp\ipykernel_11140\2196199081.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_corpus['pain_blurriness'] = corpus['pain'].append(corpus['blurriness'])


In [23]:
for coloumn in new_corpus:
    sum = 0
    count = 0
    for sentence1, sentence2 in itertools.combinations(new_corpus[coloumn], 2):
        sum += compare(sentence1, sentence2)
        count += 1

    print(coloumn, ': ', sum/count)

door_ladybug :  0.20338345897667945
pain_blurriness :  0.08946832984968557
